## Example

In [3]:
import pandas as pd
import numpy as np

######################## PARAMTERS STARTING HERE ################################
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
print("N:", N, "T:", T)

# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

print("I_0:", I_0)
print("D:", D)

# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
print("I:", I)

# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
J = 3 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")


C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "F": np.array([100, 80, 50]),
    "C": 2750,
}
V = np.zeros([N])
V_C = 30
for i in range(N):
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]

print("C:", C)
print("V:", V)
T_lead = np.array([1, 2, 3]) # T_j

######################## PARAMTERS ENDING HERE ##################################

N: 10 T: 6
I_0: [800. 600. 425. 350. 400. 524. 453. 218. 673. 200.]
D: [[138.  55. 172. 194.  94. 185.]
 [190. 101.  68. 185.  13. 136.]
 [ 79. 179.  21.  49. 199. 200.]
 [142. 103.  78. 131. 146. 155.]
 [ 35.  62.  83.  90. 197.  49.]
 [ 91.  95. 107. 127. 116. 183.]
 [105. 164.  19. 116. 119. 175.]
 [ 37. 155.  10.  77. 168.  32.]
 [108. 185. 188. 176.  81. 172.]
 [ 46. 178. 162. 200. 154. 199.]]
I: [[  0.   0.   0.   0.   0.   0.]
 [ 48.   0.   0.   0.   0.   0.]
 [  0.  20.   0.   0.   0.   0.]
 [153.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 18.  23.   0.   0.   0.   0.]
 [ 28.  45.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [109.  34.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
C: {'H': array([100.,  40., 180., 180.,  40., 180., 140., 100., 180., 140.]), 'P': array([5000., 2000., 9000., 9000., 2000., 9000., 7000., 5000., 9000.,
       7000.]), 'V': array([[44., 18.,  0.],
       [89., 45.,  0.],
       [86., 38.,  0.],
       [91., 46., 

##Solve IP

In [4]:
import gurobipy as gp
from gurobipy import GRB
import time

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

start_time = time.time()
# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()
end_time = time.time()
IP_computation_time = end_time - start_time


Set parameter MIPGap to value 0


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
MIPGap  0

Optimize a model with 398 rows, 264 columns and 838 nonzeros
Model fingerprint: 0xbe75289a
Variable types: 240 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 1.967757e+07
Presolve removed 365 rows and 122 columns
Presolve time: 0.00s
Presolved: 33 rows, 142 columns, 300 nonzeros
Found heuristic solution: objective 1.697770e+07
Variable types: 128 continuous, 14 integer (11 binary)

Root relaxation: objective 1.688731e+07, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

In [5]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    print("\nHolding cost:",holding_cost.getValue())
    print("\npurchasing and shipping cost:",purchasing_and_shipping_cost.getValue())
    print("\nContainer_cost:",container_cost.getValue())
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")

print(IP_computation_time)


Optimal objective value: 16890367.02040816

Order quantities (x_ijt):

Holding cost: 1637440.0

purchasing and shipping cost: 15239177.020408163

Container_cost: 13750.0
x[8,3,1] = 61.0
x[10,1,1] = 24.0
x[10,2,1] = 162.0
x[10,3,1] = 200.0
x[3,3,2] = 82.0
x[4,3,2] = 97.0
x[5,3,2] = 67.0
x[8,3,2] = 168.0
x[10,3,2] = 36.44897959183673
x[1,3,3] = 38.0
x[2,3,3] = 45.0
x[3,3,3] = 200.0
x[4,3,3] = 155.0
x[5,3,3] = 49.0
x[6,3,3] = 154.0
x[7,3,3] = 172.0
x[8,3,3] = 32.0
x[9,3,3] = 94.0
x[10,2,3] = 117.55102040816327
x[10,3,3] = 199.0

Ending inventory (v_it):
v[1,1] = 662.0
v[2,1] = 458.0
v[3,1] = 346.0
v[4,1] = 361.0
v[5,1] = 365.0
v[6,1] = 451.0
v[7,1] = 376.0
v[8,1] = 181.0
v[9,1] = 674.0
v[10,1] = 178.0
v[1,2] = 607.0
v[2,2] = 357.0
v[3,2] = 187.0
v[4,2] = 258.0
v[5,2] = 303.0
v[6,2] = 379.0
v[7,2] = 257.0
v[8,2] = 26.0
v[9,2] = 523.0
v[10,2] = 162.0
v[1,3] = 435.0
v[2,3] = 289.0
v[3,3] = 166.0
v[4,3] = 180.0
v[5,3] = 220.0
v[6,3] = 272.0
v[7,3] = 238.0
v[8,3] = 77.0
v[9,3] = 335.0
v[10,3]

## Solve heuristic

In [6]:
#Main heuristic

import numpy as np

def heuristic_shipping_optimizer(N, T, C, in_transit, inventory,  forecast, V, container_capacity):
    T_lead = [1, 2, 3]
    plan = {0: np.zeros((N, T)), 1: np.zeros((N, T)), 2: np.zeros((N, T))}
    shortages = np.zeros((N, T))
    leftover_volumes = np.zeros(T)
    full_containers = np.zeros(T)
    already_aggregated = np.zeros((N, T))
    leftover_product = np.zeros((N, T))

    holding_costs = C["H"]
    shipping_fixed = C["F"]
    shipping_variable = C["V"]
    container_cost = C["C"]
    purchasing_cost = C["P"]

    # Step 1: Calculate shortages (use only initial inventory + in_transit)
    I_remain = np.zeros((N, T + 1))
    for i in range(N):
        I_remain[i, 0] = inventory[i]  # 修正為一維初始庫存

    for t in range(T):
        for i in range(N):
            demand = forecast[i, t]

            available = I_remain[i, t]
            shortage = max(0, demand - available)
            shortages[i, t] = shortage

            next_inventory = max(0, available - demand) + in_transit[i, t]
            I_remain[i, t + 1] = next_inventory

    # Handle early-month shortages
    for t in range(T):
        for i in range(N):
            if shortages[i, t] > 0:
                if t - T_lead[1] < 0 and t - T_lead[2] < 0:
                    if t - T_lead[0] >= 0:
                        qty = shortages[i, t]
                        plan[0][i, t - T_lead[0]] += qty
                        shortages[i, t] = 0
                elif t - T_lead[1] >= 0 and t - T_lead[2] < 0:
                    qty = shortages[i, t]
                    plan[1][i, t - T_lead[1]] += qty
                    shortages[i, t] = 0

    # Step 2: Allocate full containers using ocean
    for t in range(T):
        air_cost_per_m3 = [(i, shipping_variable[i, 1] / V[i]) for i in range(N) if shortages[i, t] > 0 and t - T_lead[2] >= 0]
        air_cost_per_m3.sort(key=lambda x: x[1], reverse=True)

        vol_accum = sum(shortages[i, t] * V[i] for i, _ in air_cost_per_m3)
        full = np.floor(vol_accum / container_capacity)
        leftover_volume = vol_accum - full * container_capacity
        leftover_volumes[t] = leftover_volume
        full_containers[t] = full

        vol_used = 0
        for i, _ in air_cost_per_m3:
            qty = shortages[i, t]
            vol = qty * V[i]
            
            available_vol = full * container_capacity - vol_used
            if available_vol <= 0:
                leftover_product[i, t] = qty
                continue
                
            if vol <= available_vol:
                plan[2][i, t - T_lead[2]] += qty
                shortages[i, t] = 0
                vol_used += vol
            else:
                qty_fit = available_vol / V[i]
                if qty_fit > 0:
                    plan[2][i, t - T_lead[2]] += qty_fit
                    shortages[i, t] -= qty_fit
                    leftover_product[i, t] = shortages[i, t]
                    vol_used += qty_fit * V[i]

    # Step 3: Aggregate leftover if beneficial
    for t in range(T - 1):
        if leftover_volumes[t] > 0 and t - T_lead[2] >= 0:
            combined_vol = leftover_volumes[t] + leftover_volumes[t + 1]
            if combined_vol <= container_capacity:
                saved_cost = container_cost
                holding_cost = sum(holding_costs[i] * leftover_product[i, t + 1] for i in range(N))

                ocean_cost = shipping_fixed[2] + container_cost
                air_cost = sum(leftover_product[i, t] *  shipping_variable[i, 1] for i in range(N)) + sum(leftover_product[i, t+1] *  shipping_variable[i, 1] for i in range(N)) + 2*shipping_fixed[1] 

                if saved_cost > holding_cost and ocean_cost < air_cost:
                    for i in range(N):
                        leftover_product[i, t] += leftover_product[i, t + 1]
                        plan[2][i, t - T_lead[2]] += leftover_product[i, t]
                        leftover_product[i, t + 1] = 0
                    leftover_volumes[t] = combined_vol
                    leftover_volumes[t + 1] = 0
                    already_aggregated[:, t] = 1
                    full_containers[t] += 1  # aggregated container

    # Step 4: Decide how to transport leftover volumes
    for t in range(T):
        if np.all(already_aggregated[:, t] == 0) and leftover_volumes[t] > 0 and t - T_lead[2] >= 0:
            vol = leftover_volumes[t]
            containers_needed = np.ceil(vol / container_capacity)

            ocean_cost = shipping_fixed[2] + containers_needed * container_cost
            air_cost = shipping_fixed[1] + sum(shipping_variable[i, 1] * leftover_product[i, t] for i in range(N))

            method = 1 if air_cost < ocean_cost else 2
            for i in range(N):
                qty = leftover_product[i, t]
                if qty > 0 and t - T_lead[method] >= 0:
                    plan[method][i, t - T_lead[method]] += qty
                    shortages[i, t] = 0
            leftover_volumes[t] = 0

            if method == 2:
                full_containers[t] += 1  # unaggregated ocean shipment

    # Step 5: Calculate total cost
    total_holding_cost = 0
    # 計算每個月到達商品
    arrive = np.zeros((N, T + 1))
    for k in range(3):  # 三種運輸方式
        for i in range(N):
            for t in range(T):
                arrival_month = t + T_lead[k] - 1
                if arrival_month < T:
                    arrive[i, arrival_month] += plan[k][i, t]

    #計算每個月holding cost            
    end_stock = np.zeros((N, T + 1))
    for t in range(0,T): #第二個月與以後
        for i in range(N):
            if t == 0:
                end_stock[i,0] = inventory[i] - forecast[i,t] + in_transit[i,t] + arrive[i,t]
                total_holding_cost += end_stock[i,0] * C["H"][i]
            else:
                end_stock[i,t] = end_stock[i,t-1] - forecast[i,t] + in_transit[i,t] + arrive[i,t]
                total_holding_cost += end_stock[i,t] * C["H"][i]


    total_shipping_cost = 0
    total_purchasing_cost = 0
    total_container_cost = 0
    shipping_methods_used = {0: set(), 1: set(), 2: set()}

    # Express & Air: calculate purchasing and shipping
    for k in [0, 1]:
        for t in range(T):
            method_volume = sum(plan[k][i, t] for i in range(N))
            if method_volume > 0:
                shipping_methods_used[k].add(t)
            for i in range(N):
                qty = plan[k][i, t]
                total_purchasing_cost += qty * purchasing_cost[i]
                total_shipping_cost += qty * shipping_variable[i, k]

    # Ocean: only purchasing cost（
    for t in range(T):
        method_volume = sum(plan[2][i, t] for i in range(N)) 
        if method_volume > 0:
            shipping_methods_used[2].add(t)
        for i in range(N):
            qty = plan[2][i, t]
            total_purchasing_cost += qty * purchasing_cost[i]

    # Fixed cost for all shipping modes
    for k in range(3):
        total_shipping_cost += len(shipping_methods_used[k]) * shipping_fixed[k]

    # Ocean container cost
    total_container_cost += np.sum(full_containers) * container_cost

    total_cost = total_holding_cost + total_shipping_cost + total_purchasing_cost + total_container_cost

    return plan, total_cost, total_holding_cost, total_shipping_cost, total_purchasing_cost, total_container_cost


In [7]:
import time

start_time = time.time()
heuristic_obj= heuristic_shipping_optimizer(N,T,C,I,I_0,D,V,V_C)
end_time = time.time()
heuristic_computation_time = end_time - start_time

print(heuristic_obj)
print(heuristic_computation_time)

({0: array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [24.,  0.,  0.,  0.,  0.,  0.]]), 1: array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.     